In [ ]:
# CELL 1: INSTALL PACKAGES
# ============================================================================

print("Installing packages...")

# FIX: Uninstall and reinstall numpy to avoid binary incompatibility
!pip uninstall -y numpy
!pip install numpy==1.24.3

# Install other packages (remove version pins that might conflict)
!pip install -q langchain==0.1.20 langchain-community langchain-huggingface
!pip install -q transformers accelerate bitsandbytes sentence-transformers
!pip install -q chromadb sqlalchemy faiss-cpu

print("✅ Installation complete!")
print("🔄 IMPORTANT: You MUST restart runtime now!")
print("   Go to: Runtime → Restart runtime")
print("   Then run Cell 2 onwards\n")

Installing packages...
Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
  Using cached numpy-1.24.3.tar.gz (10.9 MB)
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but yo

In [ ]:
# CELL 2: IMPORTS
# ============================================================================

import urllib.request
import sqlite3

# LangChain imports
from langchain_community.utilities import SQLDatabase
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.chains import LLMChain
from langchain_huggingface import HuggingFacePipeline

# Transformers
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

print("✅ Imports successful!\n")



✅ Imports successful!



In [ ]:
# CELL 3: DOWNLOAD DATABASE & CREATE LANGCHAIN SQLDATABASE
# ============================================================================

print("Setting up database...")

# Download Chinook
url = "https://github.com/lerocha/chinook-database/raw/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite"
urllib.request.urlretrieve(url, "chinook.db")

# Create LangChain SQLDatabase (automatic schema extraction!)
db = SQLDatabase.from_uri("sqlite:///chinook.db")

print(f"✅ Database ready!")
print(f"   Tables: {db.get_usable_table_names()}\n")


Setting up database...
✅ Database ready!
   Tables: ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']



In [ ]:
# CELL 4: CREATE VECTOR STORE WITH LANGCHAIN (RAG)
# ============================================================================

print("Creating vector store...")

# Get schema info using LangChain
table_info = db.get_table_info()

# Create documents (one per table)
from langchain.schema import Document

docs = []
for table in db.get_usable_table_names():
    # Get table schema
    conn = sqlite3.connect("chinook.db")
    cursor = conn.cursor()
    cursor.execute(f"PRAGMA table_info({table})")
    cols = cursor.fetchall()
    conn.close()

    # Create document
    text = f"Table: {table}\n"
    text += "Columns: " + ", ".join([f"{col[1]} ({col[2]})" for col in cols])

    docs.append(Document(page_content=text, metadata={"table": table}))

# Create FAISS vector store with LangChain
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, embeddings)

# Create retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

print(f"✅ Vector store created with {len(docs)} documents!")

# Test retrieval
test_docs = retriever.get_relevant_documents("customers and invoices")
print(f"   Test: Found {len(test_docs)} relevant tables\n")


Creating vector store...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Vector store created with 11 documents!
   Test: Found 3 relevant tables



/usr/local/lib/python3.12/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [ ]:
# CELL 5: CREATE LANGCHAIN PROMPT TEMPLATE
# ============================================================================

print("Creating LangChain prompt template...")

# Few-shot examples
examples = [
    {
        "question": "List all artists",
        "answer": "SELECT * FROM Artist LIMIT 10;"
    },
    {
        "question": "Top 5 customers by spending",
        "answer": """SELECT c.FirstName, c.LastName, SUM(i.Total) as Total
FROM Customer c
JOIN Invoice i ON c.CustomerId = i.CustomerId
GROUP BY c.CustomerId
ORDER BY Total DESC
LIMIT 5;"""
    },
    {
        "question": "Most popular genres",
        "answer": """SELECT g.Name, COUNT(t.TrackId) as Count
FROM Genre g
JOIN Track t ON g.GenreId = t.GenreId
GROUP BY g.GenreId
ORDER BY Count DESC;"""
    }
]

# Example template
example_template = """
Question: {question}
SQL: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template=example_template
)

# Main prompt
prefix = """You are a SQLite expert. Generate a syntactically correct SQLite query.

Here are examples:"""

suffix = """
Database Schema:
{schema}

Question: {question}
SQL:"""

# Create FewShotPromptTemplate (LangChain!)
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["schema", "question"],
    example_separator="\n"
)

print("✅ LangChain prompt template ready!\n")


Creating LangChain prompt template...
✅ LangChain prompt template ready!



In [ ]:
# CELL 6: LOAD MODELS (5-10 MINUTES)
# ============================================================================

print("Loading models (takes 5-10 minutes)...\n")

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

# Quantization
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

# --- Load SQLCoder ---
print("\n📥 Loading SQLCoder...")
try:
    sqlcoder_tokenizer = AutoTokenizer.from_pretrained("defog/sqlcoder-7b-2")
    sqlcoder_model = AutoModelForCausalLM.from_pretrained(
        "defog/sqlcoder-7b-2",
        quantization_config=quant_config,
        device_map="auto",
        trust_remote_code=True
    )

    # Create pipeline and wrap with LangChain
    sqlcoder_pipe = pipeline(
        "text-generation",
        model=sqlcoder_model,
        tokenizer=sqlcoder_tokenizer,
        max_new_tokens=300,
        temperature=0.1
    )
    sqlcoder_llm = HuggingFacePipeline(pipeline=sqlcoder_pipe)

    print("✅ SQLCoder loaded & wrapped in LangChain!")
except Exception as e:
    print(f"❌ SQLCoder failed")
    sqlcoder_llm = None

# --- Load Llama/CodeLlama ---
print("\n📥 Loading Llama...")
llama_models = ["codellama/CodeLlama-7b-Instruct-hf", "mistralai/Mistral-7B-Instruct-v0.2"]
llama_llm = None
llama_name = None

for model_name in llama_models:
    try:
        print(f"   Trying {model_name}...")
        llama_tokenizer = AutoTokenizer.from_pretrained(model_name)
        llama_model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=quant_config,
            device_map="auto"
        )

        # Create pipeline and wrap with LangChain
        llama_pipe = pipeline(
            "text-generation",
            model=llama_model,
            tokenizer=llama_tokenizer,
            max_new_tokens=300,
            temperature=0.1
        )
        llama_llm = HuggingFacePipeline(pipeline=llama_pipe)
        llama_name = model_name.split('/')[-1]

        print(f"✅ {llama_name} loaded & wrapped in LangChain!")
        break
    except:
        continue

print("\n✅ Models ready!\n")



Loading models (takes 5-10 minutes)...

Device: cuda

📥 Loading SQLCoder...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0


✅ CodeLlama-7b-Instruct-hf loaded & wrapped in LangChain!

✅ Models ready!



In [ ]:
# CELL 7: CREATE LANGCHAIN CHAINS
# ============================================================================

print("Creating LangChain chains...")

# Create chains (LangChain handles everything!)
sqlcoder_chain = LLMChain(
    llm=sqlcoder_llm,
    prompt=few_shot_prompt
) if sqlcoder_llm else None

llama_chain = LLMChain(
    llm=llama_llm,
    prompt=few_shot_prompt
) if llama_llm else None

print("✅ LangChain chains created!\n")

Creating LangChain chains...
✅ LangChain chains created!



/usr/local/lib/python3.12/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [ ]:
# CELL 8: MAIN FUNCTION USING LANGCHAIN
# ============================================================================

def ask_question(question):
    """
    Main function using LangChain components:
    - Retriever (RAG)
    - Chains (LLM)
    - SQLDatabase (execution)
    """
    print(f"\n{'='*60}")
    print(f"📝 Question: {question}")
    print('='*60)

    # Step 1: Retrieve schema using LangChain retriever
    relevant_docs = retriever.get_relevant_documents(question)
    schema = "\n".join([doc.page_content for doc in relevant_docs])

    # Step 2: Generate SQL with SQLCoder using LangChain chain
    if sqlcoder_chain:
        print("\n🤖 SQLCoder (via LangChain):")
        try:
            response = sqlcoder_chain.run(schema=schema, question=question)

            # Extract SQL
            if "SQL:" in response:
                sql = response.split("SQL:")[-1].strip()
            else:
                sql = response.strip()
            sql = sql.split('\n\n')[0].strip()

            print(f"   SQL: {sql}")

            # Execute using LangChain SQLDatabase
            try:
                result = db.run(sql)
                print(f"   ✅ Success! Result: {str(result)[:100]}")
            except Exception as e:
                print(f"   ❌ Error: {str(e)[:100]}")
        except Exception as e:
            print(f"   ❌ Generation failed: {str(e)[:100]}")

    # Step 3: Generate SQL with Llama using LangChain chain
    if llama_chain:
        print(f"\n🤖 {llama_name} (via LangChain):")
        try:
            response = llama_chain.run(schema=schema, question=question)

            # Extract SQL
            if "SQL:" in response:
                sql = response.split("SQL:")[-1].strip()
            else:
                sql = response.strip()
            sql = sql.split('\n\n')[0].strip()

            print(f"   SQL: {sql}")

            # Execute using LangChain SQLDatabase
            try:
                result = db.run(sql)
                print(f"   ✅ Success! Result: {str(result)[:100]}")
            except Exception as e:
                print(f"   ❌ Error: {str(e)[:100]}")
        except Exception as e:
            print(f"   ❌ Generation failed: {str(e)[:100]}")

print("✅ Main function ready (using LangChain)!\n")


✅ Main function ready (using LangChain)!



In [ ]:
# CELL 9: TEST QUERIES
# ============================================================================

print("="*60)
print("TESTING MODELS")
print("="*60)

test_questions = [
    "List the top 5 artists with most albums",
    "Show me all customers from Canada",
    "What is the total revenue?",
    "Find the longest track",
    "Which employee has the most customers?"
]

for q in test_questions:
    ask_question(q)

print("\n" + "="*60)
print("🎉 PROJECT COMPLETE!")
print("="*60)
print("\n✅ This project uses LangChain:")
print("   - FewShotPromptTemplate for prompts")
print("   - HuggingFacePipeline for model wrapping")
print("   - LLMChain for orchestration")
print("   - FAISS for vector store")
print("   - SQLDatabase for DB operations")
print("\nTry your own:")
print("ask_question('Show me all rock albums')")
print("ask_question('Which genre has most tracks?')")


TESTING MODELS

📝 Question: List the top 5 artists with most albums

🤖 SQLCoder (via LangChain):


/usr/local/lib/python3.12/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


   SQL: SELECT a."Name", COUNT(a.AlbumId) AS AlbumCount FROM Artist a GROUP BY a.ArtistId ORDER BY AlbumCount DESC LIMIT 5;
   ❌ Error: (sqlite3.OperationalError) no such column: a.AlbumId
[SQL: SELECT a."Name", COUNT(a.AlbumId) AS Albu

🤖 CodeLlama-7b-Instruct-hf (via LangChain):


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


   SQL: SELECT g
   ❌ Error: (sqlite3.OperationalError) no such column: g
[SQL: SELECT g]
(Background on this error at: https://s

📝 Question: Show me all customers from Canada

🤖 SQLCoder (via LangChain):


...

   SQL: SELECT MAX(t.Milliseconds) AS MaxMilliseconds FROM Track t;
   ✅ Success! Result: [(5286953,)]

🤖 CodeLlama-7b-Instruct-hf (via LangChain):


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


   SQL: SELECT COUNT(*)
FROM Track
   ✅ Success! Result: [(3503,)]

📝 Question: Which employee has the most customers?

🤖 SQLCoder (via LangChain):


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


   SQL: SELECT e.FirstName, e.LastName, COUNT(DISTINCT c.CustomerId) AS customer_count FROM Employee e JOIN Customer c ON e.EmployeeId = c.SupportRepId GROUP BY e.FirstName, e.LastName ORDER BY customer_count DESC LIMIT 1;
   ✅ Success! Result: [('Jane', 'Peacock', 21)]

🤖 CodeLlama-7b-Instruct-hf (via LangChain):
   SQL: SELECT e.LastName, COUNT(c.CustomerId) as Count
FROM Employee e
JOIN
   ❌ Error: (sqlite3.OperationalError) incomplete input
[SQL: SELECT e.LastName, COUNT(c.CustomerId) as Count
FR

🎉 PROJECT COMPLETE!

✅ This project uses LangChain:
   - FewShotPromptTemplate for prompts
   - HuggingFacePipeline for model wrapping
   - LLMChain for orchestration
   - FAISS for vector store
   - SQLDatabase for DB operations

Try your own:
ask_question('Show me all rock albums')
ask_question('Which genre has most tracks?')


In [ ]:
#CELL 10: Enhanced Error Handling & Query Validation
import re

def extract_sql_from_response(response):
    """Extract only the SQL query from model response"""
    # Try to find SQL after the last "SQL:" marker
    if "SQL:" in response:
        parts = response.split("SQL:")
        sql = parts[-1].strip()

        # Remove everything after the semicolon
        if ';' in sql:
            sql = sql.split(';')[0] + ';'

        # Remove any trailing text after newlines
        sql = sql.split('\n')[0].strip()

        return sql

    return response.strip()

def validate_sql_query(sql_query):
    """Validate SQL query for safety"""
    dangerous_keywords = ['DROP', 'DELETE', 'TRUNCATE', 'ALTER', 'UPDATE']
    sql_upper = sql_query.upper()

    for keyword in dangerous_keywords:
        if keyword in sql_upper:
            return False, f"Dangerous keyword detected: {keyword}"

    if not sql_query.strip().endswith(';'):
        sql_query += ';'

    return True, sql_query

def ask_question_enhanced(question, model_name="SQLCoder"):
    """Production-ready version with error handling"""
    print(f"\n{'='*60}")
    print(f"📝 Question: {question}")
    print(f"{'='*60}\n")

    try:
        # Get schema
        schema = db.get_table_info()

        # Generate SQL
        if model_name == "SQLCoder":
            raw_response = sqlcoder_chain.run(question=question, schema=schema)
        else:
            raw_response = llama_chain.run(question=question, schema=schema)

        # Extract only the SQL query from response
        sql_query = extract_sql_from_response(raw_response)

        # Validate query
        is_valid, result = validate_sql_query(sql_query)
        if not is_valid:
            print(f"❌ Validation Failed: {result}")
            return None

        sql_query = result
        print(f"🤖 {model_name}:")
        print(f"   SQL: {sql_query}")

        # Execute query
        query_result = db.run(sql_query)
        print(f"   ✅ Success! Result: {query_result}")

        return {
            "success": True,
            "question": question,
            "sql": sql_query,
            "result": query_result
        }

    except Exception as e:
        print(f"   ❌ Error: {str(e)}")
        return {
            "success": False,
            "question": question,
            "sql": sql_query if 'sql_query' in locals() else "N/A",
            "error": str(e)
        }

In [ ]:
#CELL 11: Comprehensive Test Suite
# Expanded test cases for thorough evaluation
comprehensive_tests = [
    # Basic queries
    "Show me all customers from Canada",
    "What is the total revenue?",
    "Find the longest track",

    # Aggregations
    "How many tracks are there in total?",
    "What is the average invoice total?",

    # JOINs
    "Which employee has the most customers?",
    "List top 5 artists with most albums",
    "Show all albums by AC/DC",

    # WHERE clauses with operators
    "Find all invoices greater than $10",
    "Show tracks longer than 5 minutes",

    # Complex business queries
    "Which genre generates the most revenue?",
    "What are the top 3 countries by number of customers?",
    "Find the customer who spent the most money",

    # Edge cases
    "List tracks with no composer",
    "Show invoices from 2009"
]

print("="*60)
print("🧪 RUNNING COMPREHENSIVE TEST SUITE")
print("="*60)

# Test both models
test_results = {
    "SQLCoder": [],
    "CodeLlama": []
}

for question in comprehensive_tests:
    # Test SQLCoder
    result_sqlcoder = ask_question_enhanced(question, "SQLCoder")
    test_results["SQLCoder"].append(result_sqlcoder)

    # Test CodeLlama
    result_codellama = ask_question_enhanced(question, "CodeLlama")
    test_results["CodeLlama"].append(result_codellama)

print("\n" + "="*60)
print("📊 TEST RESULTS SUMMARY")
print("="*60)

for model_name, results in test_results.items():
    passed = sum(1 for r in results if r and r.get('success'))
    total = len(results)
    percentage = (passed/total)*100 if total > 0 else 0

    print(f"\n{model_name}:")
    print(f"   ✅ Passed: {passed}/{total} ({percentage:.1f}%)")
    print(f"   ❌ Failed: {total-passed}/{total}")

print("\n" + "="*60)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🧪 RUNNING COMPREHENSIVE TEST SUITE

📝 Question: Show me all customers from Canada



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🤖 SQLCoder:
   SQL: SELECT c.FirstName, c.LastName FROM Customer c WHERE c.Country = 'Canada';
   ✅ Success! Result: [('François', 'Tremblay'), ('Mark', 'Philips'), ('Jennifer', 'Peterson'), ('Robert', 'Brown'), ('Edward', 'Francis'), ('Martha', 'Silk'), ('Aaron', 'Mitchell'), ('Ellie', 'Sullivan')]

📝 Question: Show me all customers from Canada



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

🤖 SQLCoder:
   SQL: SELECT t.Name FROM Track t WHERE t.Composer IS NULL;
   ✅ Success! Result: [('Desafinado',), ('Garota De Ipanema',), ('Samba De Uma Nota Só (One Note Samba)',), ('Por Causa De Você',), ('Ligia',), ('Fotografia',), ('Dindi (Dindi)',), ('Se Todos Fossem Iguais A Você (Instrumental)',), ('Falando De Amor',), ('Angela',), ('Corcovado (Quiet Nights Of Quiet Stars)',), ('Outra Vez',), ('O Boto (Bôto)',), ('Canta, Canta Mais',), ('Intro/ Low Down',), ('13 Years Of Grief',), ('Stronger Than Death',), ('All For You',), ('Super Terrorizer',), ('Phoney Smile Fake Hellos',), ('Lost My Better Half',), ('Bored To Tears',), ('A.N.D.R.O.T.A.Z.',), ('Born To Booze',), ('World Of Trouble',), ('No More Tears',), ('The Begining... At Last',), ('Heart Of Gold',), ('Snowblind',), ('Like A Bird',), ('Blood In The Wall',), ('The Beginning...At Last',), ('Black Sabbath',), ('The Wizard',), ('Behind The Wall Of Sleep',), ('N.I.B.',), ('Evil Woman',), ('Sleeping Village',), ('Warning',), ('Sm

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🤖 CodeLlama:
   SQL: SELECT * FROM Track WHERE MediaTypeId IS NULL;
   ✅ Success! Result: 

📝 Question: Show invoices from 2009



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🤖 SQLCoder:
   SQL: SELECT i.InvoiceId, i.InvoiceDate, c.FirstName, c.LastName, SUM(i.Total) AS Total FROM Invoice i JOIN Customer c ON i.CustomerId = c.CustomerId WHERE EXTRACT(YEAR FROM i.InvoiceDate) = 2009 GROUP BY i.InvoiceId, i.InvoiceDate, c.FirstName, c.LastName ORDER BY i.InvoiceId NULLS LAST;
   ❌ Error: (sqlite3.OperationalError) near "FROM": syntax error
[SQL: SELECT i.InvoiceId, i.InvoiceDate, c.FirstName, c.LastName, SUM(i.Total) AS Total FROM Invoice i JOIN Customer c ON i.CustomerId = c.CustomerId WHERE EXTRACT(YEAR FROM i.InvoiceDate) = 2009 GROUP BY i.InvoiceId, i.InvoiceDate, c.FirstName, c.LastName ORDER BY i.InvoiceId NULLS LAST;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

📝 Question: Show invoices from 2009

🤖 CodeLlama:
   SQL: SELECT * FROM Track WHERE ArtistId = 1 AND Genre;
   ❌ Error: (sqlite3.OperationalError) no such column: ArtistId
[SQL: SELECT * FROM Track WHERE ArtistId = 1 AND Genre;]
(Background on this error at: https://sqlalche.me

In [ ]:
#CELL 12: Performance Metrics & Model Comparison
import time
from datetime import datetime

def benchmark_models(test_questions):
    """Compare models with performance metrics"""
    results = {
        "SQLCoder": {"success": 0, "failed": 0, "total_time": 0},
        "CodeLlama": {"success": 0, "failed": 0, "total_time": 0}
    }

    print("\n" + "="*60)
    print("⚡ PERFORMANCE BENCHMARK")
    print("="*60)

    for model in ["SQLCoder", "CodeLlama"]:
        print(f"\n🔬 Testing {model}...\n")

        for question in test_questions:
            start_time = time.time()
            result = ask_question_enhanced(question, model)
            execution_time = time.time() - start_time

            results[model]["total_time"] += execution_time

            if result and result.get('success'):
                results[model]["success"] += 1
            else:
                results[model]["failed"] += 1

    # Display comparison
    print("\n" + "="*60)
    print("🏆 MODEL COMPARISON DASHBOARD")
    print("="*60)

    for model, stats in results.items():
        total = stats["success"] + stats["failed"]
        success_rate = (stats["success"]/total)*100 if total > 0 else 0
        avg_time = stats["total_time"]/total if total > 0 else 0

        print(f"\n{model}:")
        print(f"   Success Rate: {success_rate:.1f}% ({stats['success']}/{total})")
        print(f"   Avg Time: {avg_time:.2f}s")
        print(f"   Total Time: {stats['total_time']:.2f}s")

    # Determine winner
    sqlcoder_rate = (results["SQLCoder"]["success"]/(results["SQLCoder"]["success"]+results["SQLCoder"]["failed"]))*100
    codellama_rate = (results["CodeLlama"]["success"]/(results["CodeLlama"]["success"]+results["CodeLlama"]["failed"]))*100

    winner = "SQLCoder" if sqlcoder_rate > codellama_rate else "CodeLlama"
    print(f"\n🥇 Winner: {winner}")
    print("="*60)

# Run benchmark on subset of questions
benchmark_questions = [
    "What is the total revenue?",
    "Show me all customers from Canada",
    "Which employee has the most customers?",
    "Find the longest track",
    "List top 5 artists with most albums"
]

benchmark_models(benchmark_questions)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



⚡ PERFORMANCE BENCHMARK

🔬 Testing SQLCoder...


📝 Question: What is the total revenue?



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🤖 SQLCoder:
   SQL: SELECT SUM(i.Total) AS total_revenue FROM Invoice i;
   ✅ Success! Result: [(2328.600000000004,)]

📝 Question: Show me all customers from Canada



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

🤖 CodeLlama:
   SQL: SELECT * FROM Customer;
   ✅ Success! Result: [(1, 'Luís', 'Gonçalves', 'Embraer - Empresa Brasileira de Aeronáutica S.A.', 'Av. Brigadeiro Faria Lima, 2170', 'São José dos Campos', 'SP', 'Brazil', '12227-000', '+55 (12) 3923-5555', '+55 (12) 3923-5566', 'luisg@embraer.com.br', 3), (2, 'Leonie', 'Köhler', None, 'Theodor-Heuss-Straße 34', 'Stuttgart', None, 'Germany', '70174', '+49 0711 2842222', None, 'leonekohler@surfeu.de', 5), (3, 'François', 'Tremblay', None, '1498 rue Bélanger', 'Montréal', 'QC', 'Canada', 'H2G 1A7', '+1 (514) 721-4711', None, 'ftremblay@gmail.com', 3), (4, 'Bjørn', 'Hansen', None, 'Ullevålsveien 14', 'Oslo', None, 'Norway', '0171', '+47 22 44 22 22', None, 'bjorn.hansen@yahoo.no', 4), (5, 'František', 'Wichterlová', 'JetBrains s.r.o.', 'Klanova 9/506', 'Prague', None, 'Czech Republic', '14700', '+420 2 4172 5555', '+420 2 4172 5555', 'frantisekw@jetbrains.com', 4), (6, 'Helena', 'Holý', None, 'Rilská 3174/6', 'Prague', None, 'Czech Republic',

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🤖 CodeLlama:
   SQL: SELECT e.FirstName, e.LastName, COUNT(;
   ❌ Error: (sqlite3.OperationalError) near ";": syntax error
[SQL: SELECT e.FirstName, e.LastName, COUNT(;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

📝 Question: Find the longest track



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🤖 CodeLlama:
   SQL: SELECT t.Name, COUNT(il.InvoiceLineId) as Count;
   ❌ Error: (sqlite3.OperationalError) no such column: t.Name
[SQL: SELECT t.Name, COUNT(il.InvoiceLineId) as Count;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

📝 Question: List top 5 artists with most albums

🤖 CodeLlama:
   SQL: SELECT * FROM Album WHERE ReleaseDate BETWEEN '2021-01-01 00:00:00' AND '2021-12-31 2;
   ❌ Error: (sqlite3.OperationalError) unrecognized token: "'2021-12-31 2;"
[SQL: SELECT * FROM Album WHERE ReleaseDate BETWEEN '2021-01-01 00:00:00' AND '2021-12-31 2;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

🏆 MODEL COMPARISON DASHBOARD

SQLCoder:
   Success Rate: 100.0% (5/5)
   Avg Time: 4.97s
   Total Time: 24.86s

CodeLlama:
   Success Rate: 20.0% (1/5)
   Avg Time: 23.43s
   Total Time: 117.14s

🥇 Winner: SQLCoder


In [ ]:
!pip install -U gradio langchain langchain-community langchain-huggingface transformers accelerate bitsandbytes chromadb sqlalchemy huggingface_hub


  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_huggingface-1.0.0-py3-none-any.whl.metadata (2.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.1/467.1 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.3/397.3 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.

In [ ]:
import sqlite3
import pandas as pd
import gradio as gr

def ask_question(user_db, question):
    # connect to uploaded DB
    conn = sqlite3.connect(user_db.name)
    cursor = conn.cursor()

    # Generate SQL
    prompt = f"Translate this question into an SQLite query:\n{question}\nSQL:"
    sql_query = sqlcoder_llm(prompt)

    try:
        cursor.execute(sql_query)
        rows = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        df = pd.DataFrame(rows, columns=columns)
        conn.close()
        return sql_query, df
    except Exception as e:
        conn.close()
        return f"❌ Error executing query: {e}", None

demo = gr.Interface(
    fn=ask_question,
    inputs=[
        gr.File(label="Upload SQLite Database (.db)"),
        gr.Textbox(label="Ask your question")
    ],
    outputs=[
        gr.Textbox(label="Generated SQL Query"),
        gr.Dataframe(label="Query Result")
    ],
    title="🧠 Text-to-SQL on Your Own Database",
    description="Upload your SQLite database and ask natural language questions."
)

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://adc3b8cd838dae43e0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
%%writefile requirements.txt
gradio
langchain
langchain-community
langchain-huggingface
transformers
accelerate
bitsandbytes
chromadb
sqlalchemy
huggingface_hub
pandas


Writing requirements.txt


In [ ]:
%%writefile README.md
# 🧠 Text-to-SQL Chatbot (with SQLCoder)

This app lets you **ask natural language questions** about your database — and get results instantly.
Just **upload any `.db` or `.sqlite` file**, and the model will:

1. Convert your question to an SQL query
2. Run it on the uploaded database
3. Display both the generated SQL and output table

### 🚀 Model
Powered by **SQLCoder** (Open Source LLM fine-tuned for Text-to-SQL).

### 🛠️ Tech Stack
- LangChain
- Hugging Face Transformers
- Gradio
- SQLite + Pandas

### 💡 Example
**Question:** "Show all invoices from 2009"
**Output:** SQL query + Table of results

---

👨‍💻 Built with ❤️ using Gradio and LangChain.


Writing README.md


In [ ]:
%%writefile app.py
import sqlite3
import pandas as pd
import gradio as gr
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# ============================================================
# 🚀 Load SQLCoder model
# ============================================================
model_id = "defog/sqlcoder-7b-2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="auto"
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    do_sample=False
)

sqlcoder_llm = HuggingFacePipeline(pipeline=pipe)

# ============================================================
# 🧠 Define query function
# ============================================================
def ask_question(user_db, question):
    """Takes an uploaded SQLite database + a question, returns SQL + result"""
    if not user_db:
        return "❌ Please upload a database file.", None

    conn = sqlite3.connect(user_db.name)
    cursor = conn.cursor()

    # Create a Text-to-SQL prompt
    prompt = f"Translate this question into an SQLite query:\nQuestion: {question}\nSQL:"
    sql_query = sqlcoder_llm(prompt)

    try:
        cursor.execute(sql_query)
        rows = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        df = pd.DataFrame(rows, columns=columns)
        conn.close()
        return sql_query, df
    except Exception as e:
        conn.close()
        return f"❌ Error executing query: {e}", None

# ============================================================
# 🎨 Gradio UI
# ============================================================
demo = gr.Interface(
    fn=ask_question,
    inputs=[
        gr.File(label="Upload SQLite Database (.db)"),
        gr.Textbox(label="Ask your question")
    ],
    outputs=[
        gr.Textbox(label="Generated SQL Query"),
        gr.Dataframe(label="Query Result")
    ],
    title="🧠 Text-to-SQL on Your Own Database",
    description="Upload your SQLite database and ask natural language questions."
)

if __name__ == "__main__":
    demo.launch()


Writing app.py


In [ ]:
!wget https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite -O Chinook.db


--2025-10-28 09:28:19--  https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1007616 (984K) [application/octet-stream]
Saving to: ‘Chinook.db’

Chinook.db          100%[===================>] 984.00K  --.-KB/s    in 0.03s   

2025-10-28 09:28:20 (37.8 MB/s) - ‘Chinook.db’ saved [1007616/1007616]

